# Classifying MNIST with a Nengo Ensemble

As a fun experiment, this notebook describes how to use a single, quite large neural ensemble in Nengo to perform an interesting classification task (namely, classifying images of handwritten digits from the MNIST database). Nengo ensembles, to explain, are collections of spiking neurons that respond selectively to different regions in an implicit representational space. This response selectivity is formally captured by assignment of a "preferred direction" of each neuron, which is a vector that identifies a representational state that triggers maximal spiking activity in the neuron. If the number of neurons is considerably larger than the dimensionality of the represenational space, and the preferred direction vectors (or 'encoders') are suitably distributed throughout the space, then it is possible to use linear regression to predict the values of arbitrary functions of the ensemble's representational state from the spiking activity of the ensemble's neurons. This works because the regression seeks out a set of linear weights in the very high-dimensional neuron space (where each neuron can be thought of as a feature that takes on different values depending on its input) that approximate a non-linear function in the lower-dimensional representational space. 

To demonstrate this use of linear weights to compute non-linear functions in context of image classification in nengo, we'll first load the images and perform some basic preprocessing to normalize the image vectors and convert the target classifications into binary vectors:

In [6]:
import numpy as np
import cPickle

with open('data/mnist.pkl', 'rb') as data:
    train_set, cv_set, test_set = cPickle.load(data)

def preprocess(data):
    xs = data[0]
    ys = data[1]
    xs_norms = np.linalg.norm(xs, axis=1)
    xs = np.divide(xs, xs_norms[:, np.newaxis]) 
    return xs, ys    

def binarize(data):
    indices = [tuple(data),tuple(range(len(data)))]
    tmatrix = np.zeros((10, len(data)))
    tmatrix[indices] = 1
    return tmatrix 

train_data, train_targs = preprocess(train_set)
test_data, test_targs = preprocess(test_set)

train_targs = binarize(train_targs).T
test_targs = binarize(test_targs).T

It is now possible to define a nengo model that contains a single, very large ensemble whose decoders (i.e.the weights to used to compute a desired function from a set of neural activities) are optimized to minimize the squared error on the mapping between the training images in the MNIST dataset, and the binary vectors that indicate the correct classification decision for each of these images. In other words, the evaluation points used by nengo's decoder solver are the training images. For reasons concerning computational efficiency, only a portion of images are actually used in the example below. 

In [7]:
import nengo
from nengo.utils.connection import target_function
from nengo.solvers import LstsqL2, conjgrad

model = nengo.Network()

test_image = train_data[2345,:] # Pick an image to classify

with model:
    inp = nengo.Node(test_image)  
    ens = nengo.Ensemble(n_neurons=50000, dimensions=784, radius=1)
    out = nengo.Node(size_in=10)
    
    nengo.Connection(inp, ens)
    nengo.Connection(ens, out, solver=LstsqL2(solver=conjgrad), 
                     **target_function(train_data[:15000,:], train_targs[:15000,:]))
    
    outProbe = nengo.Probe(out)
    
sim = nengo.Simulator(model)
sim.run(0.3)

Simulation finished in 0:00:06.                                                 


To test the classifier, we can compare the probed output of the ensemble during the simulation to the target value associated with the test image. If the dimension of the probed output with the maximum value is equal to the dimension of the target output that is non-zero, the ensemble can be treated as performing a correct classification. Here are the results:

In [15]:
print 'Ensemble output: '
print sim.data[outProbe][150]
print ''
print 'Target output: '
print train_targs[2345,:]


Ensemble output: 
[ 0.17537283 -0.01310605 -0.02519724  0.09067443 -0.07917026 -0.05874371
  0.83711041 -0.03059647 -0.02662353  0.12863922]

Target output: 
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]


On this example at least, the classifier works quite well. More robust testing will be added shortly...